**Import dependencies**

In [15]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from splinter import Browser
import time
import json

**Defining functions to instantiate webdriver, opening browser and closing browser.**

In [140]:
def init_browser():
    # Import Splinter and set the chromedriver path
    executable_path = {"executable_path": "chromedriver.exe"}
    browser = Browser("chrome", **executable_path, headless=False)
    return browser

def open_browser(browser, url):
    browser.visit(url)

def close_browser(browser):
    browser.quit()

**Collecting Nasa Mars news**

In [141]:
# Visit the Nasa URL
url = "https://mars.nasa.gov/news/"
browser = init_browser()
open_browser(browser, url)

In [142]:
# use splinter to get the text
news_title = browser.find_by_xpath(xpath_news_title).first.value
news_p = browser.find_by_xpath(xpath_news).first.value

In [143]:
print(news_title)
print(news_p)

MEDLI2 Installation on Mars 2020 Aeroshell Begins
Hardware installed onto NASA's Mars 2020 entry vehicle this week will help to increase the safety of future Mars landings.


In [144]:
# Scrape the browser into a soup object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
news_title = soup.find('div', {"class" : "list_text"}).find('div', {"class" : "content_title"}).find('a').text
news_p = soup.find('div', {"class" : "list_text"}).find('div', {"class" : "article_teaser_body"}).text

In [145]:
print(news_title)
print(news_p)
close_browser(browser)

MEDLI2 Installation on Mars 2020 Aeroshell Begins
Hardware installed onto NASA's Mars 2020 entry vehicle this week will help to increase the safety of future Mars landings.


**Collecting JPL Mars Space Images - Latest Featured Image**

In [149]:
# Import Splinter and set the chromedriver path
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser = init_browser()
open_browser(browser, url)

In [150]:
try:
    time.sleep(1)
    element = browser.find_by_name('category')
    element.select('featured')
except:
    time.sleep(1)


In [152]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
imgs = soup.find('div', { 'class': 'image_and_description_container'}).find('div', {'class' : 'img'}).find_all('img')
relative_image_url = imgs[0]["src"]
featured_image_url = "https://www.jpl.nasa.gov"+relative_image_url
print(f" { featured_image_url}")
close_browser(browser)

 https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23354-640x350.jpg


**Collecting Mars Latest Weather Tweet**

In [154]:
browser = init_browser()
url = "https://twitter.com/marswxreport?lang=en"
open_browser(browser, url)
html = browser.html
close_browser(browser)

In [204]:
soup = BeautifulSoup(html, 'html.parser')

tweets = soup.find_all('li', {'class' : 'js-stream-item'})
for tweet in tweets:
    screen_name = tweet.find('div')["data-screen-name"]
    if screen_name == 'MarsWxReport':
        #print(tweet)
        result = tweet.find('div', {'class' : 'content'})\
                      .find('div', {'class' : 'js-tweet-text-container'})\
                      .find('p', {'class' : 'tweet-text'})
        mars_weather = result.text
        print(mars_weather)
        break 




InSight sol 233 (2019-07-23) low -98.8ºC (-145.9ºF) high -25.7ºC (-14.2ºF)
winds from the SE at 4.6 m/s (10.2 mph) gusting to 16.2 m/s (36.2 mph)
pressure at 7.60 hPapic.twitter.com/ksOXPg28yb


**Collecting Planet details from Mars Facts**

_Use Pandas to scrape the following site and decode the mars facts in the list._

In [4]:
url = "https://space-facts.com/mars/"
tables = pd.read_html(url)
mars_facts = tables[1]
mars_facts

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


_Cleaning up the table_

In [5]:
mars_facts.columns = ["description", "value"]

In [6]:
mars_facts = mars_facts.set_index('description')

In [7]:
mars_html_output = mars_facts.to_html("./html/mars_facts.html", justify="justify", bold_rows = True, table_id="mars_facts")
print(mars_facts)

                                              value
description                                        
Equatorial Diameter:                       6,792 km
Polar Diameter:                            6,752 km
Mass:                 6.39 × 10^23 kg (0.11 Earths)
Moons:                          2 (Phobos & Deimos)
Orbit Distance:            227,943,824 km (1.38 AU)
Orbit Period:                  687 days (1.9 years)
Surface Temperature:                   -87 to -5 °C
First Record:                     2nd millennium BC
Recorded By:                   Egyptian astronomers


**Collecting images from Mars Hemispheres.**

In [310]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser = init_browser()
open_browser(browser, url)
html = browser.html

#---------------
hemisphere_image_urls = []

soup = BeautifulSoup(html, 'html.parser')
images = soup.find('div', {'id': 'product-section'}).find_all('h3')

In [311]:
result = {}
for image in images:
    result = {}
    image_text = image.text
    print(image)
    try:
        time.sleep(10)
        browser.find_link_by_partial_text(image_text)
        browser.click_link_by_partial_text(image_text)
        time.sleep(10)
    except:
        time.sleep(1)
    title= browser.find_by_xpath('//div[@class="content"]//h2[@class="title"]').first.text
    img_url = browser.find_by_xpath('//div[@class="downloads"]//ul//li//a')["href"]
    result = { "title" : title, "img_url" : img_url}
    hemisphere_image_urls.append(result)
    browser.back()
    time.sleep(10)
close_browser(browser)
print(hemisphere_image_urls)

<h3>Cerberus Hemisphere Enhanced</h3>
<h3>Schiaparelli Hemisphere Enhanced</h3>
<h3>Syrtis Major Hemisphere Enhanced</h3>
<h3>Valles Marineris Hemisphere Enhanced</h3>
[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
